In [1]:
import tensorflow as tf
import numpy as np
import os
import sys
from math import log

Let's read load the annotation TFRecords to get the annotations as well as the number of rows and columns to better understand the shape of our data.

In [2]:
annotation_file_name = 'annotations.tfrecords'
genotype_file_name = 'cases.tfrecords'

num_epochs = 2
X_dim   = 60508 # This is how long our vector is.
                # 4322 genotypes with  14 annotations each = 60508
mb_size = 10    # Minibatch size
lr =      0.01  # Learning rate

gt_length = 4322 #Use in places that have 784
h_dim = 100      #use where you see 100

annotations = tf.placeholder(tf.int64, shape=[None, X_dim], name='annotations')
ncols = tf.placeholder(tf.int64, shape=[None, 1], name='ncols')
nrows = tf.placeholder(tf.int64, shape=[None, 1], name='nrows')

def read_and_decode(filename_queue):
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(
        serialized_example,
        features={'annotations': tf.FixedLenFeature([X_dim], tf.int64),
              'ncols': tf.FixedLenFeature([1], tf.int64),
              'nrows': tf.FixedLenFeature([1], tf.int64),
              })
    annotations = tf.cast(features['annotations'], tf.int32, name='anno_casting')
    ncols = tf.cast(features['ncols'], tf.int32, name='ncol_casting')
    nrows = tf.cast(features['nrows'], tf.int32, name='nrow_casting')
    return annotations,ncols,nrows


def get_all_records():
    with tf.Session() as sess:
        filename_queue = tf.train.string_input_producer([annotation_file_name])
        init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
        sess.run(init_op)
        annotations,ncols,nrows = read_and_decode(filename_queue)
        
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
        try:
            annotations,ncols,nrows = sess.run([annotations,ncols,nrows])
            return annotations,ncols,nrows

        except tf.errors.OutOfRangeError as e:
            coord.request_stop(e)
        finally:
            coord.request_stop()
            coord.join(threads)

annotations,ncols,nrows = get_all_records()

# convert list to scalars where appropriate
ncols,nrows = ncols[0],nrows[0]

annotations = np.reshape(annotations, (nrows,ncols))
print(annotations.shape)

(4322, 14)


Cool.  Now I have a numpy array of annotations as well as the number of variants in my model (n=4322).  I can use these features in the next step.

## Building a generator
I want to build a generator that takes as input an h_dim-dimensional vector and returns a nrows-dimensional vector of 0, 1, or 2 that correspond to reference, heterozygous, and homozygous alternate, respectively. 

Before we do that, let's define some variables.

In [3]:
# Generator variables
# genotypes can only be 0, 1 or 2
min_value = 0.   
max_value = 3.

# Descriminator variables


In [4]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev,dtype=tf.float32)

def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])


# Set variables
# Generator Net

Z = tf.placeholder(tf.float32, shape=[None, h_dim])

G_W1 = tf.Variable(xavier_init([h_dim, 128]),dtype=tf.float32)
G_b1 = tf.Variable(tf.zeros(shape=[128]),dtype=tf.float32)

G_W2 = tf.Variable(xavier_init([128, gt_length]))
G_b2 = tf.Variable(tf.zeros(shape=[gt_length]))

theta_G = [G_W1, G_W2, G_b1, G_b2]

def sample_z(m, n):
    return np.random.uniform(-1., 1., size=[m, n]).tolist()

#def generator(z):
#    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
#    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    # Strech values from [0,3)
#    G_scaled = tf.div(
#       tf.subtract(
#          G_log_prob, min_value
#       ), 
#        (max_value - min_value)
#    )
#    # round to integer
#    G_int = tf.cast(tf.round(G_scaled),tf.int32)
#    G_transposed = tf.transpose(G_int)
#    return(G_transposed)

def generator(z):
    with tf.name_scope('generator'):
        G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
        G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
        G_prob = tf.nn.sigmoid(G_log_prob)

        return tf.cast(G_prob,dtype=tf.float32)


generator(Z)

<tf.Tensor 'generator/Sigmoid:0' shape=(?, 4322) dtype=float32>

## Building a discriminator
Now that we have a generator, we need to build a discriminator to dtermine whether it looks realistic or not.  It needs to take in a vector of genotypes (nrows) and output a prediction of 'real' or 'fake'.

However, this is also where I want to do something a bit different from other algortihms.  I want the discriminator to have access to something the generator doesn't - the annotations.  Biologicially, this means we are effectively bounding the possible genotypes to what would be expected based on certain parameters like 'protein coding', 'missense prediction', 'allele frequency', etc.  Then, we'll let the network sort out what the appropriate genotype should be allowed for each variant.  So what I want to do is to combine the genotype input (either real or fake) with each of the annotations, and then create a neural network to predict true or false.


In [5]:

# Set variables
# Discriminator Net
X = tf.placeholder(tf.float32, shape=[None, gt_length])
#print('X: {}'.format(X))

D_W1 = tf.Variable(xavier_init([gt_length,128]))
#print('D_W1: {}'.format(D_W1))
D_b1 = tf.Variable(tf.zeros(shape=[128]))

D_W2 = tf.Variable(xavier_init([128,1]))
#print('D_W2: {}'.format(D_W2))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]

def discriminator(x):
    with tf.name_scope('Descriminator'):
        #print('X: {}'.format(x))
        #print('D_W1: {}'.format(D_W1))
        #print('D_b1: {}'.format(D_b1))
        D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
        D_logit = tf.matmul(D_h1, D_W2) + D_b2
        D_prob = tf.nn.sigmoid(D_logit)
        return D_prob, D_logit



# Read genotypes
So far we've: 
 * read through our annotations
 * built a genotype generator
 * built a discriminator that combines data from the genotypes
 
 but now we need to build a function to read in our genotypes in batches

In [6]:
def read_and_decode_genotypes(filename_queue):
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(
        serialized_example,
        features={'label':     tf.FixedLenFeature([], tf.int64),
                  'genotypes': tf.FixedLenFeature([gt_length], tf.int64),
                 }        
    )
    
    genotypes = tf.cast(features['genotypes'], tf.float32, name='geno_casting')
    genotypes = tf.Print(genotypes,[genotypes],'genotypes: ')
    genotypes = tf.reshape(genotypes,[gt_length])
    return genotypes

def inputs(genotype_file_name, mb_size=mb_size, num_epochs=num_epochs):
    if not num_epochs:
        num_epochs = None
    with tf.name_scope('input'):
        filename_queue = tf.train.string_input_producer([genotype_file_name], num_epochs=num_epochs)
        gt = read_and_decode_genotypes(filename_queue)
        #print('gt: {}'.format(gt))
        gts = tf.train.shuffle_batch(
                [gt],
                batch_size=mb_size,
                num_threads=2,
                capacity=1000 + 3 * mb_size,
                min_after_dequeue=1)
        gts = tf.Print(gts,[gts],'gts shape: ')
        print('gts: {}'.format(gts))
    return gts

In [7]:
GT = tf.placeholder(tf.float32, shape=[mb_size, gt_length])

G_sample = generator(Z)
D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(G_sample)


D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
D_loss = D_loss_real + D_loss_fake
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

#Iterate
with tf.Session() as sess:
   
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init_op)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    for it in range(10):
        GT = inputs(genotype_file_name)
        print('GT is '.format(GT))
        z_mb = sample_z(mb_size, h_dim)
        print('z_mb: {}'.format(np.array(z_mb).shape))
        
        D_real = discriminator(GT)
        D_fake = discriminator(generator(sample_z(mb_size, h_dim)))
        #print('D_real: {}'.format(D_real))
        #print('D_fake: {}'.format(D_fake))
        
        _, D_loss_curr = sess.run(
            [D_solver, D_loss],
            feed_dict={X: GT, Z: z_mb}
        )

        _, G_loss_curr = sess.run(
           [G_solver, G_loss],
           feed_dict={X: GT, Z: z_mb }
            
       )

    if it % 10 == 0:
        print('Iter: {}; D_loss: {:.4}; G_loss: {:.4}'
                 .format(it, D_loss_curr, G_loss_curr))

        samples = sess.run(G_sample, feed_dict={Z: z_mb})
    
    coord.request_stop()
    coord.join(threads)

gts: Tensor("input/Print_1:0", shape=(10, 4322), dtype=float32)
GT is 
z_mb: (10, 100)
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: input_producer/input_producer_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](input_producer, input_producer/RandomShuffle)]]


TypeError: The value of a feed cannot be a tf.Tensor object. Acceptable feed values include Python scalars, strings, lists, numpy ndarrays, or TensorHandles.